# Baseline: Pre-optimized Santa 2025 Submission

This notebook establishes our baseline using the best known pre-optimized submission.

**Target score**: 68.894234
**Best known score**: 70.647306

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree
from shapely.ops import unary_union
import json
import os

getcontext().prec = 30

# Tree vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

print("Tree shape loaded with 15 vertices")

Tree shape loaded with 15 vertices


In [2]:
class ChristmasTree:
    """Christmas tree polygon with high precision coordinates"""
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x).replace('s', ''))
        self.center_y = Decimal(str(center_y).replace('s', ''))
        self.angle = Decimal(str(angle).replace('s', ''))
        
        # Build polygon
        initial_polygon = Polygon(list(zip(TX, TY)))
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, 
            xoff=float(self.center_x), yoff=float(self.center_y))

def has_overlap(trees):
    """Check if any trees overlap"""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                # Check if it's a real overlap (not just touching)
                intersection = poly.intersection(polygons[idx])
                if intersection.area > 1e-10:
                    return True
    return False

def get_side_length(trees):
    """Get the side length of the bounding box"""
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

print("ChristmasTree class and helper functions defined")

ChristmasTree class and helper functions defined


In [3]:
# Load the best pre-optimized submission
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(baseline_path)
print(f"Loaded {len(df)} rows")
print(df.head())

Loaded 20100 rows
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  


In [4]:
# Parse the submission into tree configurations for each N
def parse_submission(df):
    """Parse submission CSV into dict of N -> list of trees"""
    configs = {}
    for _, row in df.iterrows():
        id_parts = row['id'].split('_')
        n = int(id_parts[0])
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        
        if n not in configs:
            configs[n] = []
        configs[n].append(ChristmasTree(x, y, deg))
    return configs

print("Parsing submission...")
configs = parse_submission(df)
print(f"Parsed configurations for N=1 to N={max(configs.keys())}")

Parsing submission...


Parsed configurations for N=1 to N=200


In [5]:
# Calculate score for each N and total score
def calculate_score(configs):
    """Calculate total score: sum(side^2 / N) for all N"""
    side_lengths = {}
    scores_by_n = {}
    total_score = Decimal('0')
    
    for n in sorted(configs.keys()):
        trees = configs[n]
        side = Decimal(str(get_side_length(trees)))
        side_lengths[n] = float(side)
        score_n = side ** 2 / Decimal(str(n))
        scores_by_n[n] = float(score_n)
        total_score += score_n
    
    return float(total_score), side_lengths, scores_by_n

print("Calculating score...")
total_score, side_lengths, scores_by_n = calculate_score(configs)
print(f"\nTotal Score: {total_score:.6f}")
print(f"Target Score: 68.894234")
print(f"Gap: {total_score - 68.894234:.6f} ({(total_score - 68.894234) / 68.894234 * 100:.2f}%)")

Calculating score...



Total Score: 70.676102
Target Score: 68.894234
Gap: 1.781868 (2.59%)


In [6]:
# Validate no overlaps (sample a few N values)
print("Validating no overlaps...")
test_ns = [1, 2, 5, 10, 20, 50, 100, 150, 200]
for n in test_ns:
    if n in configs:
        if has_overlap(configs[n]):
            print(f"WARNING: N={n} has overlapping trees!")
        else:
            print(f"N={n}: OK (no overlaps)")

Validating no overlaps...
N=1: OK (no overlaps)
N=2: OK (no overlaps)
N=5: OK (no overlaps)
N=10: OK (no overlaps)
N=20: OK (no overlaps)
N=50: OK (no overlaps)
N=100: OK (no overlaps)
N=150: OK (no overlaps)
N=200: OK (no overlaps)


In [7]:
# Show score breakdown by N ranges
print("\nScore breakdown by N ranges:")
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores_by_n[n] for n in range(start, end+1) if n in scores_by_n)
    print(f"N={start}-{end}: {range_score:.4f}")

print(f"\nTop 10 N values by score contribution:")
sorted_scores = sorted(scores_by_n.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:10]:
    print(f"  N={n}: {score:.4f} (side={side_lengths[n]:.4f})")


Score breakdown by N ranges:
N=1-10: 4.3291
N=11-50: 14.7130
N=51-100: 17.6411
N=101-150: 17.1441
N=151-200: 16.8487

Top 10 N values by score contribution:
  N=1: 0.6612 (side=0.8132)
  N=2: 0.4508 (side=0.9495)
  N=3: 0.4347 (side=1.1420)
  N=5: 0.4168 (side=1.4437)
  N=4: 0.4165 (side=1.2908)
  N=7: 0.3999 (side=1.6731)
  N=6: 0.3996 (side=1.5484)
  N=9: 0.3874 (side=1.8673)
  N=8: 0.3854 (side=1.7559)
  N=15: 0.3792 (side=2.3850)


In [ ]:
# Copy baseline submission to /home/submission/
import shutil
shutil.copy(baseline_path, '/home/submission/submission.csv')
print(f"Copied baseline to /home/submission/submission.csv")

# Save metrics
metrics = {
    'cv_score': total_score,
    'target_score': 68.894234,
    'gap': total_score - 68.894234,
    'gap_percent': (total_score - 68.894234) / 68.894234 * 100
}

with open('/home/code/experiments/001_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nMetrics saved to experiments/001_baseline/metrics.json")
print(f"\nBaseline established: {total_score:.6f}")